In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("multiturn.csv")

In [3]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv("OPEN_API_KEY")
openai.api_key = API_KEY
client = openai

In [4]:
records = df.values[0].tolist()

In [5]:
records

['Write me a sentence with 25 words ', 'Count the words', nan]

In [6]:
questions = [x for x in records if x is not None and not (isinstance(x, float) and np.isnan(x))]

In [7]:
questions

['Write me a sentence with 25 words ', 'Count the words']

In [8]:
model = "gpt-4o-mini"

In [9]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv("OPEN_API_KEY")
openai.api_key = API_KEY
client = openai

def run_assistant(model, questions):
    assistant = client.beta.assistants.create(model=model, temperature=0)
    thread = client.beta.threads.create()

    for q in questions:
        client.beta.threads.messages.create(thread_id=thread.id, role="user", content=q)
        client.beta.threads.runs.create_and_poll(thread_id=thread.id, assistant_id=assistant.id)

    messages = client.beta.threads.messages.list(thread_id=thread.id)

    message_history = []
    
    for message in messages.data[::-1]:
        role = message.role
        content = message.content[0].text.value
        message_history.append({
            "role": role.capitalize(),
            "content": content
        })

    return message_history

In [10]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
load_dotenv()

def run_langchain_api(model, questions):
    
    chat = ChatOpenAI(model=model, temperature=0)
    prompt = ChatPromptTemplate.from_messages([
        ("placeholder", "{messages}")]
    )
    chain = prompt | chat
    chat_history = ChatMessageHistory()

    for q in questions:
        chat_history.add_user_message(q)
        response = chain.invoke({"messages": chat_history.messages})
        chat_history.add_ai_message(response.content)  # Use response.content directly

    message_history = []

    for chat in chat_history.messages:
        if isinstance(chat, HumanMessage):
            role = "User"
        elif isinstance(chat, AIMessage):
            role = "Assistant"

        message_history.append({
            "role": role.capitalize(),
            "content": chat.content
        })

    return message_history

In [11]:
assistant_messages = run_assistant(model, questions)

In [12]:
langchain_messages = run_langchain_api(model, questions)

In [13]:
assistant_messages

[{'role': 'User', 'content': 'Write me a sentence with 25 words '},
 {'role': 'Assistant',
  'content': 'In the quiet of the evening, as the sun dipped below the horizon, the stars began to twinkle, creating a magical atmosphere filled with wonder and peace.'},
 {'role': 'User', 'content': 'Count the words'},
 {'role': 'Assistant',
  'content': "I apologize for the oversight. The sentence I provided contains 26 words. Here’s a revised sentence with exactly 25 words:\n\nAs the sun set behind the mountains, the sky transformed into a canvas of vibrant colors, inspiring everyone to pause and appreciate nature's breathtaking beauty."}]

In [14]:
langchain_messages

[{'role': 'User', 'content': 'Write me a sentence with 25 words '},
 {'role': 'Assistant',
  'content': 'In the quiet of the evening, as the sun dipped below the horizon, the stars began to twinkle, casting a magical glow over the peaceful landscape.'},
 {'role': 'User', 'content': 'Count the words'},
 {'role': 'Assistant',
  'content': "I apologize for the oversight. The sentence I provided contains 26 words. Here’s a revised sentence with exactly 25 words:\n\nAs the sun set behind the mountains, the sky transformed into a canvas of vibrant colors, inspiring everyone to pause and appreciate nature's breathtaking beauty."}]